In [ ]:
# Install and import necessary R libraries for this notebook
install.packages('plotly')

library(dplyr)
library(tidyr)
library(ggplot2)
library(bigrquery)
library(plotly)

In [ ]:
# Setup BigQuery project, create 'bqml_tutorial' dataset in project if necessary
# ENTER YOUR PROJECT ID HERE
PROJECT_ID <- 'gcp-data-science-demo'

bq_auth(use_oob = TRUE)

bq_dataset_obj <- bq_dataset(PROJECT_ID, 'bqml_tutorial')

project_dataset_name <- paste0(bq_dataset_obj$project, '.', 
  bq_dataset_obj$dataset)

# If 'bqml_tutorial' dataset does not exist in project, create it
if(bq_dataset_exists(bq_dataset_obj))
{
  cat(paste0('Dataset ', project_dataset_name, ' already exists\n'))
} else {
  cat(paste0('Dataset ', project_dataset_name, ' does not exist\n'))
  
  bq_dataset_create(bq_dataset_obj)
  
  cat(paste0('Created dataset ', project_dataset_name, '\n'))
}

In [ ]:
sales_by_category_month_query <- "
  SELECT
    FORMAT_DATE('%Y-%m', date) AS month,
    category,
    category_name,

    SUM(volume_sold_gallons) AS category_monthly_gallons,
    SUM(sale_dollars) AS category_monthly_dollars

  FROM
    `bigquery-public-data.iowa_liquor_sales.sales` Sales

  WHERE
    # Remove current month so as to avoid partial data
    FORMAT_DATE('%Y-%m', date) < FORMAT_DATE('%Y-%m', CURRENT_DATE())

  GROUP BY
    month, category, category_name

  ORDER BY
    category_monthly_gallons DESC
  "

sales_by_category_month <- bq_project_query(
  x = PROJECT_ID,
  query = sales_by_category_month_query
  ) %>%
  bq_table_download()

sales_by_category_month

In [ ]:
# Get top overall sales categories across entire time span
NUM_TOP_CATEGORIES <- 20

top_overall_sales_categories <- sales_by_category_month %>%
  group_by(category, category_name) %>%
  summarize(
    total_gallons = sum(category_monthly_gallons),
    total_dollars = sum(category_monthly_dollars)
    ) %>%
  ungroup() %>%
  arrange(desc(total_gallons)) %>%
  head(n = NUM_TOP_CATEGORIES) %>%
  print()

In [ ]:
# Plot monthly volume for top categories using plotly

top_sales_categories_by_month <- sales_by_category_month %>%
  filter(
    category_name %in%  
      top_overall_sales_categories$category_name
    ) %>%
  mutate(
    month = as.Date(paste0(month, '-01'))
    ) %>%
  arrange(month, category_name)

top_sales_categories_by_month_plot <- ggplot( 
  data = top_sales_categories_by_month,
  aes(
    x = month,
    y = category_monthly_gallons,
    color = category_name
    ),
  ) + 
  scale_x_date(date_breaks = '1 year', date_labels = '%Y') + 
  geom_point() + 
  geom_path()

ggplotly(top_sales_categories_by_month_plot)

In [ ]:
bigquery_params <- list(
  # Start w/ September 2016 since plot above shows category shift
  start_date = '2016-09-01'
  )

In [ ]:
# Look at correlation among categories' volume across months

correlation_among_categories_query <- "
    WITH
    MonthlyTotals AS
    (
      SELECT
        FORMAT_DATE('%Y-%m', date) AS month,
        SUM(volume_sold_gallons) AS total_monthly_volume

      FROM
        `bigquery-public-data.iowa_liquor_sales.sales`

      WHERE
        # Start w/ date given by query parameter
        date >= @start_date AND
        # Remove current month so as to avoid partial data
        FORMAT_DATE('%Y-%m', date) < FORMAT_DATE('%Y-%m', CURRENT_DATE())

      GROUP BY
        month
    ),

    MonthCategory AS
    (
      SELECT
        FORMAT_DATE('%Y-%m', date) AS month,
        category,
        category_name,

        SUM(volume_sold_gallons) AS category_monthly_volume,

        SAFE_DIVIDE(
          SUM(volume_sold_gallons),
          total_monthly_volume
          ) * 100 AS category_pct_of_month_volume

      FROM
        `bigquery-public-data.iowa_liquor_sales.sales` Sales

      LEFT JOIN
        MonthlyTotals ON 
          FORMAT_DATE('%Y-%m', Sales.date) = MonthlyTotals.month

      WHERE
        # Start w/ date given by query parameter
        date >= @start_date AND    
        # Remove current month so as to avoid partial data
        FORMAT_DATE('%Y-%m', date) < FORMAT_DATE('%Y-%m', CURRENT_DATE())

      GROUP BY
        month, category, category_name, total_monthly_volume
    )

    SELECT
      Category1.category AS category1,
      Category1.category_name AS category_name1,

      Category2.category AS category2,
      Category2.category_name AS category_name2,

      COUNT(DISTINCT Category1.month) AS num_months,

      CORR(
        Category1.category_pct_of_month_volume,
        Category2.category_pct_of_month_volume
        ) AS category_corr_across_months,

      AVG(Category1.category_pct_of_month_volume) AS
        category1_avg_pct_of_month_volume,
      AVG(Category2.category_pct_of_month_volume) AS
        category2_avg_pct_of_month_volume

    FROM
      MonthCategory Category1

    INNER JOIN
      MonthCategory Category2 ON
      (
        Category1.month = Category2.month
      )

    GROUP BY
      category1, category_name1, category2, category_name2

    HAVING
      # At least 2 years' worth of overlapping months
      num_months >= 24 AND
      # Each category accounts for >= 1% of monthly volume (on average)
      category1_avg_pct_of_month_volume >= 1 AND
      category2_avg_pct_of_month_volume >= 1

    ORDER BY
      category_corr_across_months
    "

correlation_among_categories <- bq_project_query(
  x = PROJECT_ID,
  query = correlation_among_categories_query,
  parameters = bigquery_params
  ) %>%
  bq_table_download()

correlation_among_categories

In [ ]:
# OPTIONAL - show heat map of correlation coefficients
category_correlation_heatmap <- correlation_among_categories %>%
  mutate(
    rounded_category_corr = round(category_corr_across_months, 2)
    ) %>%
  plot_ly(
    data = .,
    x = ~category_name1, 
    y = ~category_name2,
    z = ~category_corr_across_months,
    text = ~rounded_category_corr,
    type = 'heatmap'
    ) %>%
  add_annotations(
    x = ~category_name1, 
    y = ~category_name2,
    text = ~rounded_category_corr,
    showarrow = FALSE
    )

category_correlation_heatmap

In [ ]:
# Look at monthly sales volume by category
chosen_categories <- c('Cream Liqueurs', 'American Brandies', 
  'American Schnapps', 'Flavored Rum')

chosen_categories_sales_by_month <- sales_by_category_month %>%
  filter(                                 
    month >= '2016-09',
    category_name %in% chosen_categories
    ) %>%
  mutate(
    month = as.Date(paste0(month, '-01'))
    ) %>%
  arrange(category_name, month)

chosen_categories_sales_by_month_plot <- ggplot( 
  data = chosen_categories_sales_by_month,
  aes(
    x = month,
    y = category_monthly_gallons,
    color = category_name
    ),
  ) + 
  scale_x_date(date_breaks = '1 year', date_labels = '%Y') + 
  geom_point() + 
  geom_path()

ggplotly(chosen_categories_sales_by_month_plot)

In [ ]:
create_iowa_liquor_sales_category_info_query <- "
  CREATE OR REPLACE TABLE bqml_tutorial.iowa_liquor_sales_category_info AS
    (
      SELECT
        category_name,
        REGEXP_EXTRACT_ALL(LOWER(category_name), '[a-z]+') AS cat_tokens,

        item_description,
        REGEXP_EXTRACT_ALL(LOWER(item_description), '[a-z]+') AS item_tokens,

        SUM(volume_sold_gallons) total_item_volume,

        SAFE_DIVIDE(
          SUM(sale_dollars),
          SUM(volume_sold_gallons)
          ) AS avg_dollars_per_gallon,

        SAFE_DIVIDE(
          SUM(IF(FORMAT_DATE('%m', date) IN ('10', '11', '12'), 
            volume_sold_gallons, 0)),
          SUM(volume_sold_gallons)
          ) * 100 AS pct_item_volume_Q4

      FROM
        `bigquery-public-data.iowa_liquor_sales.sales` Sales

      WHERE
        category_name IS NOT NULL AND
        item_description IS NOT NULL

      GROUP BY
        category_name, item_description

      HAVING
        total_item_volume > 0

      ORDER BY
        total_item_volume DESC
    )
    "

bq_project_query(
  x = PROJECT_ID,
  query = create_iowa_liquor_sales_category_info_query
  )

iowa_liquor_sales_category_info <- bq_project_query(
  x = PROJECT_ID,
  query = "
    SELECT *  
    FROM bqml_tutorial.iowa_liquor_sales_category_info
    ORDER BY total_item_volume DESC
    "
  ) %>%
  bq_table_download()

iowa_liquor_sales_category_info

In [ ]:
create_iowa_liquor_sales_kmeans_clustering_query <- "
    CREATE OR REPLACE MODEL bqml_tutorial.category_names_kmeans10
      OPTIONS(model_type = 'kmeans', num_clusters = 10, 
        kmeans_init_method = 'KMEANS++') AS
    (
      SELECT
        cat_tokens,
        item_tokens,
        avg_dollars_per_gallon,
        pct_item_volume_Q4

      FROM
        bqml_tutorial.iowa_liquor_sales_category_info
    )
    "

bq_project_query(
  x = PROJECT_ID,
  query = create_iowa_liquor_sales_kmeans_clustering_query
  )

iowa_liquor_sales_kmeans_clustering_eval <- bq_project_query(
  x = PROJECT_ID,
  query = "
    SELECT * 
    FROM ML.EVALUATE(MODEL bqml_tutorial.category_names_kmeans10)
    "
  ) %>%
  bq_table_download()

iowa_liquor_sales_kmeans_clustering_eval

In [ ]:
iowa_liquor_sales_kmeans_clustering_centroids <- bq_project_query(
  x = PROJECT_ID,
  query = "
    SELECT *
    FROM ML.CENTROIDS(MODEL bqml_tutorial.category_names_kmeans10)
    "
  ) %>%
  bq_table_download() %>%
  print()

In [ ]:
iowa_liquor_sales_kmeans_clustering_results_query <- "
    # CREATE OR REPLACE TABLE bqml_tutorial.category_names_kmeans_results AS
    SELECT
      CENTROID_ID AS cluster,

      SUM(total_item_volume) AS total_volume,

      SAFE_DIVIDE(
        SUM(total_item_volume * avg_dollars_per_gallon),
        SUM(total_item_volume)
        ) AS avg_dollars_per_gallon,

      SAFE_DIVIDE(
        SUM(total_item_volume * pct_item_volume_Q4),
        SUM(total_item_volume)
        ) AS pct_volume_Q4,

      STRING_AGG(item_description, ', ' ORDER BY total_item_volume DESC LIMIT 10)
        AS top_items,

      STRING_AGG(DISTINCT category_name, ', ' ORDER BY category_name
        ) AS categories

    FROM
      ML.PREDICT(
        MODEL bqml_tutorial.category_names_kmeans10,
        TABLE bqml_tutorial.iowa_liquor_sales_category_info
      )

    GROUP BY
      cluster

    ORDER BY
      total_volume DESC
    "

iowa_liquor_sales_kmeans_clustering_results <- bq_project_query(
  x = PROJECT_ID,
  query = iowa_liquor_sales_kmeans_clustering_results_query
  ) %>%
  bq_table_download()

iowa_liquor_sales_kmeans_clustering_results 